In [1]:
import asyncio
import threading
import time
import concurrent.futures

# I/O Bound Use Cases

## Async IO Bound example

In [2]:
async def fetch_data_async(task_id, delay): 
    print(f"Async task {task_id}: starting..")
    await asyncio.sleep(delay) # Non-blocking io simulation
    print(f"Async task {task_id}: done")
    return f"Data {task_id}"

async def run_async(): 
    start = time.time()
    tasks = [fetch_data_async(i,1) for i in range(5)]
    results = await asyncio.gather(*tasks)
    elapsed = time.time() - start
    print(f"Async total time: {elapsed:.2f} seconds")
    return results

In [3]:
results = await run_async()

Async task 0: starting..
Async task 1: starting..
Async task 2: starting..
Async task 3: starting..
Async task 4: starting..
Async task 0: done
Async task 1: done
Async task 2: done
Async task 3: done
Async task 4: done
Async total time: 1.00 seconds


## Threading I/O-bound example

In [8]:
def fetch_data_thread(task_id, delay): 
    print(f"Thread {task_id}: starting")
    time.sleep(delay) # Blocking I/O
    print(f"Thread {task_id}: done")
    return f"Data {task_id}"

def run_threading(): 
    start = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor: 
        futures = [executor.submit(fetch_data_thread, i, 1) for i in range(5)]
        results = [f.result() for f in concurrent.futures.as_completed(futures)]
    elapsed = time.time() - start
    print(f"Threading total time: {elapsed:.2f} seconds")

In [9]:
run_threading()

Thread 0: startingThread 1: starting

Thread 2: starting
Thread 3: starting
Thread 4: starting
Thread 1: doneThread 3: done
Thread 4: done
Thread 0: done

Thread 2: done
Threading total time: 1.01 seconds


# Asyncio is slightly better for IO

- Memory: 1 thread vs 5 threads = less RAM
- Context switching: Asyncio switches when tasks say they aare done (cooperative). Threads get interrupted by OS (preemptive), this costs more
- No GIL flights: Threads compete for Python's Global interpreter lock. Asyncio doesn't since it is singlethreaded

# CPU-bound Use Cases

In [15]:
def cpu_task(n): 
    """CPU intensive calculatio"""
    count = 0
    for i in range(n): 
        count += i **2
    return count

async def cpu_task_async(n): 
    """Same CPU work but async"""
    count = 0
    for i in range(n): 
        count += i ** 2
    return count

## Threading cpu-bound use

In [16]:
start = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers = 2) as executor:
    results = list(executor.map(cpu_task, [100_000_000, 100_000_000]))
print(f"Threading CPU time: {time.time() - start:.2f}s")

Threading CPU time: 43.15s


## Async CPU-bound use

In [17]:
async def run_async_cpu(): 
    start = time.time()
    results = await asyncio.gather(
        cpu_task_async(100_000_000), 
        cpu_task_async(100_000_000) 
    )
    print(f"Async CPU time: {time.time() - start:.2f}s")


In [18]:
await run_async_cpu()

Async CPU time: 43.11s


### Wait. This is not what I expected. 

It's because Aysyncio avoided thread creation overhead and context switching costs

The hidden costs threading pays:
1. GIL contention: Threads fight over the lock constantly
2. OS context switches: ~1-10 microseconds per switch
3. Thread stack: Each thread reserves ~8MB memory even if idle
4. Cache thrashing: Switching threads invalidates CPU cache
Asyncio has none of these costs because it's single-threaded. It just runs one task, then the next.

In [ ]:
import multiprocessing

start = time.time()
with multiprocessing.Pool(2) as pool:
  results = pool.map(cpu_task, [10_000_000, 10_000_000])
print(f"Multiprocessing time: {time.time() - start:.2f}s")

## Why did the above fail? - Multiprocessing pickles your function to send it to workers, but Jupyter's __main__ module isn't available in spawned processes.
 - macOS and Windows use spawn to create child processes (not fork like Linux)
 - With spawn, child processes need to import the cpu_task function
 - In Jupyter notebooks, the __main__ module is the notebook itself, and child processes can't access functions defined in it
  
  

In [5]:
from tasks import cpu_task
from multiprocessing import Pool
import time

In [6]:
start = time.time()
with Pool(2) as p: 
    results = p.map(cpu_task, [100_000_000, 100_000_000])
print(f"Time : {time.time() - start:.2f}s")

Time : 21.91s


# Multiprocessing > Asycnio > Threading 